In [32]:
import requests
from pprint import PrettyPrinter
import tensorflow as tf
import requests
import json
import base64


pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/spam-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1694556015'}]}


In [33]:
def prepare_json_for_serving(text):
    feature_spec = {
        "Message": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = {
        "examples": {
            "b64": base64.b64encode(example).decode()
        }
    }
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": [result]
    })

In [34]:
def predict_text_spam_or_ham(text):
    json_data = prepare_json_for_serving(text)
    
    endpoint = "http://localhost:8080/v1/models/spam-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return "Text spam" if prediction > 0.5 else "Text HAM"

In [36]:
text_spam = [
    "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",
    "Nah I don't think he goes to usf, he lives around here though",
    "URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18",
    "Fair enough, anything going on?",
]

for text in text_spam:
    result = predict_text_spam_or_ham(text)
    print(f"Text message: '{text}' => {result}")


Text message: 'FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv' => Text HAM
Text message: 'Nah I don't think he goes to usf, he lives around here though' => Text spam
Text message: 'URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18' => Text HAM
Text message: 'Fair enough, anything going on?' => Text spam
